In [2]:
import sys
import os

folder_path = os.path.dirname(os.path.abspath(__file__)) if '__file__' in globals() else os.getcwd()

sys.path.append(os.path.abspath(os.path.join(folder_path, '..')))

from csv_analysis import *

In [2]:
df = read_first_n_rows("../data/q2/merged_openalex_works_data.csv", nrows=15)
df

,work_id,publication_year,institution_id,country,country_code,city,geonames_city_id,author_position,author_id
0,https://openalex.org/W100000002,1988,https://openalex.org/I4210095140,United States,US,Durham,4464368,first,https://openalex.org/A5012874907
1,https://openalex.org/W100000002,1988,https://openalex.org/I4210095140,United States,US,Durham,4464368,last,https://openalex.org/A5006412880
2,https://openalex.org/W1000000251,1990,https://openalex.org/I1292966370,United Kingdom,GB,London,2643743,first,https://openalex.org/A5057387418
3,https://openalex.org/W1000000251,1990,https://openalex.org/I1292966370,United Kingdom,GB,London,2643743,last,https://openalex.org/A5035275962
4,https://openalex.org/W1000000589,2010,https://openalex.org/I7877124,Germany,DE,Berlin,2950159,first,https://openalex.org/A5091270897
5,https://openalex.org/W1000000589,2010,https://openalex.org/I7877124,Germany,DE,Berlin,2950159,last,https://openalex.org/A5071995676
6,https://openalex.org/W1000001125,2001,https://openalex.org/I231086526,Poland,PL,Siedlce,759412,first,https://openalex.org/A5060029945
7,https://openalex.org/W1000001192,1988,https://openalex.org/I1328594524,United States,US,Kansas City,4393217,first,https://openalex.org/A5072333347
8,https://openalex.org/W1000001192,1988,https://openalex.org/I4210128618,United States,US,Kansas City,4273837,first,https://openalex.org/A5072333347
9,https://openalex.org/W1000001555,2011,https://openalex.org/I2802849423,United States,US,Lubbock,5525577,first,https://openalex.org/A5050684082


In [ ]:
total_records = count_records_in_csv("../data/q2/merged_openalex_works_data.csv")
print(f"Total records: {total_records}")

Total records: 364045619


 יוצר קובץ של שם כותב וכל השנים ומדינות הפרסום שלו

In [ ]:
import pandas as pd
from collections import defaultdict
from tqdm import tqdm

# נתיב לקובץ ה-CSV
file_path = '../data/q2/merged_openalex_works_data.csv'

# פונקציה לקריאה מהירה ב-Pandas על פי חתך בגודל 100,000 שורות
def process_large_csv_and_merge(file_path, output_file):
    chunk_size = 100000
    author_data = defaultdict(list)  # מילון שמאכסן רשימות של זוגות (year, country)
    
    # קריאת הקובץ כדי לחשב את מספר החתיכים (chunks)
    print("Calculating total number of lines in the file...")
    total_lines = sum(1 for _ in open(file_path))  # סופר את מספר השורות בקובץ
    total_chunks = total_lines // chunk_size + (1 if total_lines % chunk_size > 0 else 0)
    print(f"Total number of chunks: {total_chunks}")

    # שימוש ב-tqdm להציג את התקדמות הקריאה
    with tqdm(total=total_chunks, desc="Processing chunks", unit="chunk") as pbar:
        # קריאת הקובץ בחתיכות ופילוט כל חתיך
        print("Starting to process the chunks...")
        for chunk in pd.read_csv(file_path, chunksize=chunk_size):
            # עיבוד כל חתיך
            chunk = chunk[['author_id', 'publication_year', 'country']]
            
            # עובר על כל שורה ומוסיף את השנה והמדינה לרשימה של אותו כותב
            for _, row in chunk.iterrows():
                author_id = row['author_id']
                year_country_pair = (row['publication_year'], row['country'])
                
                # מוסיף את הזוג לרשימה של הכותב במילון
                author_data[author_id].append(year_country_pair)
            
            # עדכון התקדמות עבור כל חתיכה
            pbar.update(1)

    # יצירת DataFrame ושמירת התוצאה
    print("Finished processing all chunks. Creating final DataFrame...")
    author_data_list = [(author_id, year_country_pairs) for author_id, year_country_pairs in author_data.items()]
    result_df = pd.DataFrame(author_data_list, columns=['author_id', 'year_country_pairs'])
    
    # שמירת התוצאה לקובץ
    print(f"Saving the processed data to {output_file}...")
    result_df.to_csv(output_file, index=False)
    print(f"Data saved successfully to {output_file}")

# קריאה לפונקציה עם הנתיב לקובץ הפלט
output_file = '../data/moving_between_countries/processed_author_data.csv'
process_large_csv_and_merge(file_path, output_file)


Calculating total number of lines in the file...
Total number of chunks: 3641


Processing chunks:   0%|          | 0/3641 [00:00<?, ?chunk/s]

Starting to process the chunks...


Processing chunks: 100%|██████████| 3641/3641 [3:43:38<00:00,  3.69s/chunk]  


Finished processing all chunks. Creating final DataFrame...
Saving the processed data to data/q2/processed_author_data.csv...
Data saved successfully to data/q2/processed_author_data.csv


In [3]:
df = read_first_n_rows("../data/moving_between_countries/processed_author_data.csv", nrows=15)
df

,author_id,year_country_pairs
0,https://openalex.org/A5012874907,"[(1988, 'United States'), (1988, 'United State..."
1,https://openalex.org/A5006412880,"[(1988, 'United States'), (1979, 'United State..."
2,https://openalex.org/A5057387418,"[(1990, 'United Kingdom'), (2010, 'Germany'), ..."
3,https://openalex.org/A5035275962,"[(1990, 'United Kingdom'), (1982, 'United King..."
4,https://openalex.org/A5091270897,"[(2010, 'Germany'), (1992, 'Germany'), (1997, ..."
5,https://openalex.org/A5071995676,"[(2010, 'Germany'), (2014, 'Germany'), (2005, ..."
6,https://openalex.org/A5060029945,"[(2001, 'Poland'), (2003, 'Poland'), (2012, 'P..."
7,https://openalex.org/A5072333347,"[(1988, 'United States'), (1988, 'United State..."
8,https://openalex.org/A5050684082,"[(2011, 'United States'), (2012, 'Netherlands'..."
9,https://openalex.org/A5050464501,"[(2011, 'United States'), (2012, 'Netherlands'..."


In [ ]:
total_records = count_records_in_csv("../data/moving_between_countries/processed_author_data.csv")
print(f"Total records: {total_records}")

Total records: 32852780


יוצר קובץ שמכיל רק כותבים שהחליפו מדינה

In [ ]:
import pandas as pd
from tqdm import tqdm

# נתיב לקובץ הקלט
input_file = '../data/moving_between_countries/processed_author_data.csv'
# נתיב לקובץ הפלט
output_file = '../data/moving_between_countries/authors_with_country_changes.csv'

# קריאה לקובץ הקלט
df = pd.read_csv(input_file)

# פונקציה לבדוק אם כותב החליף מדינה
def has_changed_country(year_country_pairs):
    countries = set([country for _, country in year_country_pairs])
    return len(countries) > 1

# יצירת קובץ חדש שיכיל רק כותבים שהחליפו מדינה
filtered_authors = []

# הדפסת הודעה על תחילת הקריאה והעיבוד
print("Starting the process of filtering authors with country changes...")

# עובר על כל כותב ומסנן את כותבים שהחליפו מדינה
with tqdm(total=len(df), desc="Processing authors", unit="author") as pbar:
    for _, row in df.iterrows():
        author_id = row['author_id']
        year_country_pairs = eval(row['year_country_pairs'])  # משנה את המייצג של הרשימה לצורת רשימה אמיתית
        
        # אם הכותב החליף מדינה, נוסיף אותו לרשימה
        if has_changed_country(year_country_pairs):
            # ממיין את רשימת הזוגות לפי שנה
            sorted_pairs = sorted(year_country_pairs, key=lambda x: x[0])  # מיון לפי השנה
            filtered_authors.append([author_id, sorted_pairs])
        
        # עדכון התקדמות עבור כל כותב
        pbar.update(1)

# המרת הרשימה ל-DataFrame
filtered_df = pd.DataFrame(filtered_authors, columns=['author_id', 'year_country_pairs'])

# שמירה לקובץ חדש
print(f"Saving the processed data to {output_file}...")
filtered_df.to_csv(output_file, index=False)

print(f"Data has been saved successfully to {output_file}")


Starting the process of filtering authors with country changes...


Processing authors: 100%|██████████| 32852780/32852780 [49:23<00:00, 11086.28author/s] 


Saving the processed data to data/q2/authors_with_country_changes.csv...
Data has been saved successfully to data/q2/authors_with_country_changes.csv


In [ ]:
total_records = count_records_in_csv("../data/moving_between_countries/authors_with_country_changes.csv")
print(f"Total records: {total_records}")

Total records: 5644034


In [2]:
df = read_first_n_rows("../data/moving_between_countries/authors_with_country_changes.csv", nrows=15)
df

,author_id,year_country_pairs
0,https://openalex.org/A5012874907,"[(1976, 'United States'), (1977, 'United State..."
1,https://openalex.org/A5057387418,"[(1990, 'United Kingdom'), (1990, 'United King..."
2,https://openalex.org/A5035275962,"[(1971, 'Australia'), (1976, 'Sweden'), (1976,..."
3,https://openalex.org/A5091270897,"[(1984, 'Germany'), (1992, 'Germany'), (1994, ..."
4,https://openalex.org/A5071995676,"[(1999, 'Germany'), (1999, 'Germany'), (2001, ..."
5,https://openalex.org/A5050684082,"[(1987, 'Netherlands'), (1987, 'Netherlands'),..."
6,https://openalex.org/A5050464501,"[(2008, 'Netherlands'), (2010, 'Netherlands'),..."
7,https://openalex.org/A5022831628,"[(1997, 'United Kingdom'), (1997, 'China'), (2..."
8,https://openalex.org/A5011787061,"[(1994, 'United States'), (1997, 'United State..."
9,https://openalex.org/A5017785808,"[(1988, 'United States'), (1993, 'United State..."


## יצירת קובץ שמגדיר את כל המעברים בין מדינות


מעבר: שינוי מדינה בין שני פרסומים עוקבים של אותו חוקר, גם אם באותה שנה.

שנת המעבר: שנת הפרסום במדינת היעד.

עזיבה קבועה: A→B בשנה t, ואין פרסום במדינת A בחלון t+1 עד t+X.

עזיבה זמנית: A→B בשנה t, ויש פרסום במדינת A בחלון t+1 עד t+X.

הגעה קבועה: A→B בשנה t, ואין פרסום במדינה אחרת כלשהי בחלון t+1 עד t+X.

הגעה זמנית: A→B בשנה t, ויש פרסום במדינה אחרת כלשהי בחלון t+1 עד t+X.

פרסומים: נספרת כל רשומת (שנה, מדינה) בנתונים שלך, כלומר מספר עבודות, לא מספר חוקרים ייחודיים.

ברירת מחדל X=3 שנים, ניתן לשינוי.

In [4]:
import ast
import gc
from collections import defaultdict
from typing import List, Tuple, Optional

import pandas as pd
from tqdm import tqdm


def parse_year_country_pairs(s: str) -> List[Tuple[int, str]]:
    """
    Parse a string like "[(1976, 'United States'), (1977, 'United States')]" to [(1976,'United States'), ...]
    Returns cleaned list with int years and stripped country strings.
    """
    if s is None or s == "" or pd.isna(s):
        return []
    try:
        pairs = ast.literal_eval(s)
    except Exception:
        return []
    out = []
    for item in pairs:
        if not isinstance(item, (list, tuple)) or len(item) != 2:
            continue
        y, c = item
        if y is None or c is None:
            continue
        try:
            y = int(y)
        except Exception:
            continue
        out.append((y, str(c).strip()))
    return out


def compute_country_year_metrics_streaming(
    input_csv_path: str,
    output_csv_path: str,
    x_perm_years: int = 3,
    chunksize: int = 50_000,
) -> pd.DataFrame:
    """
    Streaming computation of yearly country metrics from a very large CSV.

    Input CSV must contain columns: author_id, year_country_pairs.

    Output CSV columns per (year, country):
      - left_permanent
      - left_temporary
      - arrived_permanent
      - arrived_temporary
      - arrived_total
      - publications
    """
    left_perm = defaultdict(int)        # (year, country_from)
    left_temp = defaultdict(int)        # (year, country_from)
    arr_perm = defaultdict(int)         # (year, country_to)
    arr_temp = defaultdict(int)         # (year, country_to)
    publications = defaultdict(int)     # (year, country)

    parsed_rows = 0
    skipped_rows = 0

    reader = pd.read_csv(
        input_csv_path,
        usecols=["author_id", "year_country_pairs"],
        dtype={"author_id": "string", "year_country_pairs": "string"},
        chunksize=chunksize,
        low_memory=True,
    )

    # progress bar without a known total
    pbar = tqdm(unit="rows", desc="Processing", dynamic_ncols=True, smoothing=0.1)

    for chunk in reader:
        for _, row in chunk.iterrows():
            pairs = parse_year_country_pairs(row["year_country_pairs"])
            if not pairs:
                skipped_rows += 1
                continue

            # publications: count every (year, country)
            for y, c in pairs:
                publications[(y, c)] += 1

            # stable sort by year, preserve original order within same year
            pairs_with_idx = list(enumerate(pairs))
            pairs_sorted = [p for _, p in sorted(pairs_with_idx, key=lambda x: (x[1][0], x[0]))]

            # deduplicate same (year, country) within author
            seen_yc = set()
            seq = []
            for y, c in pairs_sorted:
                key = (y, c)
                if key in seen_yc:
                    continue
                seen_yc.add(key)
                seq.append((y, c))

            n = len(seq)
            if n >= 2:
                for i in range(n - 1):
                    y_from, c_from = seq[i]
                    y_to, c_to = seq[i + 1]
                    if c_from == c_to:
                        continue
                    t = y_to  # transition year is target year

                    # leave classification for origin country
                    returned_to_origin_within_window = False
                    for j in range(i + 2, n):
                        y_future, c_future = seq[j]
                        if y_future > t + x_perm_years:
                            break
                        if c_future == c_from and y_future >= t + 1:
                            returned_to_origin_within_window = True
                            break
                    if returned_to_origin_within_window:
                        left_temp[(t, c_from)] += 1
                    else:
                        left_perm[(t, c_from)] += 1

                    # arrival classification for destination country
                    left_destination_within_window = False
                    for j in range(i + 2, n):
                        y_future, c_future = seq[j]
                        if y_future > t + x_perm_years:
                            break
                        if c_future != c_to and y_future >= t + 1:
                            left_destination_within_window = True
                            break
                    if left_destination_within_window:
                        arr_temp[(t, c_to)] += 1
                    else:
                        arr_perm[(t, c_to)] += 1

            parsed_rows += 1

        pbar.update(len(chunk))
        del chunk
        gc.collect()

    pbar.close()

    # build output
    keys = set()
    keys |= set(left_perm.keys())
    keys |= set(left_temp.keys())
    keys |= set(arr_perm.keys())
    keys |= set(arr_temp.keys())
    keys |= set(publications.keys())

    records = []
    for (y, c) in keys:
        ap = arr_perm.get((y, c), 0)
        at = arr_temp.get((y, c), 0)
        records.append({
            "year": y,
            "country": c,
            "left_permanent": left_perm.get((y, c), 0),
            "left_temporary": left_temp.get((y, c), 0),
            "arrived_permanent": ap,
            "arrived_temporary": at,
            "arrived_total": ap + at,
            "publications": publications.get((y, c), 0),
        })

    out = pd.DataFrame(records).sort_values(["year", "country"]).reset_index(drop=True)
    out.to_csv(output_csv_path, index=False)

    print(f"Done. Parsed rows: {parsed_rows}, skipped rows: {skipped_rows}, unique (year,country) cells: {len(keys)}")
    return out


# Example:
output_df = compute_country_year_metrics_streaming(
    input_csv_path="../data/moving_between_countries/authors_with_country_changes.csv",
    output_csv_path="../data/moving_between_countries/country_year_metrics.csv",
    x_perm_years=3,
    chunksize=50_000
)
# print(output_df.head())


Processing: 5644034rows [30:54, 3044.12rows/s] 


Done. Parsed rows: 5644034, skipped rows: 0, unique (year,country) cells: 21066


In [8]:
df = read_first_n_rows("../data/moving_between_countries/country_year_metrics.csv", nrows=1000)
df

,year,country,left_permanent,left_temporary,arrived_permanent,arrived_temporary,arrived_total,publications
0,4,China,0,0,0,0,0,4
1,4,France,0,0,0,0,0,1
2,27,France,0,0,0,0,0,4
3,29,France,0,0,0,0,0,3
4,208,Hungary,0,0,1,0,1,1
...,...,...,...,...,...,...,...,...
995,1862,Brazil,0,0,2,0,2,7
996,1862,Cameroon,0,0,1,0,1,1
997,1862,Canada,0,0,2,1,3,18
998,1862,Chile,0,0,0,0,0,1


In [3]:
total_records = count_records_in_csv("../data/moving_between_countries/country_year_metrics.csv")
print(f"Total records: {total_records}")

Total records: 21066


In [5]:
import pandas as pd

# קריאת הקובץ המלא
df = pd.read_csv("../data/moving_between_countries/country_year_metrics.csv")

# סינון הרשומות של ישראל
df_israel = df[df['country'] == 'Israel']

# הצגת התוצאה
df_israel


,year,country,left_permanent,left_temporary,arrived_permanent,arrived_temporary,arrived_total,publications
160,1802,Israel,0,0,0,0,0,1
210,1807,Israel,0,0,0,0,0,1
1394,1874,Israel,0,0,0,0,0,1
1473,1876,Israel,1,0,0,0,0,1
1601,1879,Israel,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...
20084,2020,Israel,3465,3941,2310,5165,7475,72401
20308,2021,Israel,4179,3239,2974,4780,7754,80997
20533,2022,Israel,4354,2375,3003,3427,6430,69284
20759,2023,Israel,6341,25,6187,139,6326,56166


יוצר קובץ של כל הכותבים שעברו מדינה, ואחת המדינות היא ישראל

In [ ]:
import pandas as pd

# Input and output file paths
input_file = '../data/moving_between_countries/authors_with_country_changes.csv'
output_file = '../data/moving_between_countries/authors_country_changes_in_israel.csv'

# Read the input CSV file
df = pd.read_csv(input_file)

# Function to check if the author has been in Israel
def was_in_israel(year_country_pairs):
    # Check if any of the (year, country) pairs contains 'Israel'
    return any(country == 'Israel' for _, country in year_country_pairs)

# Create a new list to hold authors who have been in Israel
filtered_authors = []

# Iterate over each author and filter those who have been in Israel
for _, row in df.iterrows():
    author_id = row['author_id']
    year_country_pairs = eval(row['year_country_pairs'])  # Convert string representation of list to actual list
    
    # If the author has been to Israel, add them to the filtered list
    if was_in_israel(year_country_pairs):
        filtered_authors.append([author_id, year_country_pairs])

# Convert the list of filtered authors to a DataFrame
filtered_df = pd.DataFrame(filtered_authors, columns=['author_id', 'year_country_pairs'])

# Save the filtered data to a new CSV file
filtered_df.to_csv(output_file, index=False)

print(f"Data has been saved to {output_file}")


Data has been saved to data/q2/authors_country_changes_in_israel.csv


In [ ]:
total_records = count_records_in_csv("../data/moving_between_countries/authors_country_changes_in_israel.csv")
print(f"Total records: {total_records}")

Total records: 109148


In [5]:
df = read_first_n_rows("../data/moving_between_countries/authors_country_changes_in_israel.csv", nrows=30)
df

,author_id,year_country_pairs
0,https://openalex.org/A5060843284,"[(1946, 'China'), (1956, 'United States'), (19..."
1,https://openalex.org/A5042966626,"[(1965, 'United States'), (1965, 'United State..."
2,https://openalex.org/A5081258957,"[(1953, 'United States'), (1954, 'United State..."
3,https://openalex.org/A5058738251,"[(1967, 'United States'), (1969, 'United State..."
4,https://openalex.org/A5085070114,"[(1982, 'South Africa'), (1982, 'South Africa'..."
5,https://openalex.org/A5055529501,"[(1989, 'Israel'), (1989, 'Israel'), (1993, 'I..."
6,https://openalex.org/A5063861158,"[(1968, 'United States'), (1969, 'United State..."
7,https://openalex.org/A5020398241,"[(1987, 'United States'), (1987, 'United State..."
8,https://openalex.org/A5026764667,"[(1971, 'United States'), (1975, 'United State..."
9,https://openalex.org/A5063463701,"[(1995, 'Germany'), (1996, 'Germany'), (1997, ..."


#  מעברים מישראל ולישראל לפי שנה


In [1]:
import pandas as pd

# Input and output file paths
input_file = '../data/moving_between_countries/country_year_metrics.csv'
output_file = '../data/moving_between_countries/transitions_to_from_israel.csv'

# קריאת הקובץ המלא
df = pd.read_csv(input_file)

# סינון הרשומות של ישראל
df_israel = df[df['country'] == 'Israel']

# הצגת התוצאה
df_israel.to_csv(output_file, index=False)


In [2]:
df_israel

,year,country,left_permanent,left_temporary,arrived_permanent,arrived_temporary,arrived_total,publications
160,1802,Israel,0,0,0,0,0,1
210,1807,Israel,0,0,0,0,0,1
1394,1874,Israel,0,0,0,0,0,1
1473,1876,Israel,1,0,0,0,0,1
1601,1879,Israel,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...
20084,2020,Israel,3465,3941,2310,5165,7475,72401
20308,2021,Israel,4179,3239,2974,4780,7754,80997
20533,2022,Israel,4354,2375,3003,3427,6430,69284
20759,2023,Israel,6341,25,6187,139,6326,56166


## יצירת קובץ קלט לתרשים מסוג Sankey 

In [1]:
import ast
import gc
from collections import defaultdict
from typing import List, Tuple

import pandas as pd
from tqdm import tqdm


def parse_year_country_pairs(s: str) -> List[Tuple[int, str]]:
    if s is None or s == "" or pd.isna(s):
        return []
    try:
        pairs = ast.literal_eval(s)
    except Exception:
        return []
    out = []
    for item in pairs:
        if not isinstance(item, (list, tuple)) or len(item) != 2:
            continue
        y, c = item
        if y is None or c is None:
            continue
        try:
            y = int(y)
        except Exception:
            continue
        out.append((y, str(c).strip()))
    return out


def compute_sankey_metrics(
    input_csv_path: str,
    output_csv_path: str,
    x_perm_years: int = 3,
    chunksize: int = 50_000,
) -> pd.DataFrame:
    """
    Compute source-target metrics for Sankey diagram, filtered for Israel.
    """
    left_perm = defaultdict(int)
    left_temp = defaultdict(int)
    arr_perm = defaultdict(int)
    arr_temp = defaultdict(int)

    parsed_rows = 0
    skipped_rows = 0

    reader = pd.read_csv(
        input_csv_path,
        usecols=["author_id", "year_country_pairs"],
        dtype={"author_id": "string", "year_country_pairs": "string"},
        chunksize=chunksize,
        low_memory=True,
    )

    pbar = tqdm(unit="rows", desc="Processing", dynamic_ncols=True, smoothing=0.1)

    for chunk in reader:
        for _, row in chunk.iterrows():
            pairs = parse_year_country_pairs(row["year_country_pairs"])
            if not pairs or len(pairs) < 2:
                skipped_rows += 1
                continue

            # stable sort by year
            pairs_with_idx = list(enumerate(pairs))
            pairs_sorted = [p for _, p in sorted(pairs_with_idx, key=lambda x: (x[1][0], x[0]))]

            # deduplicate
            seen_yc = set()
            seq = []
            for y, c in pairs_sorted:
                key = (y, c)
                if key in seen_yc:
                    continue
                seen_yc.add(key)
                seq.append((y, c))

            n = len(seq)
            if n >= 2:
                for i in range(n - 1):
                    y_from, c_from = seq[i]
                    y_to, c_to = seq[i + 1]
                    if c_from == c_to:
                        continue
                    t = y_to  # transition year

                    # leave classification
                    returned_to_origin_within_window = False
                    for j in range(i + 2, n):
                        y_future, c_future = seq[j]
                        if y_future > t + x_perm_years:
                            break
                        if c_future == c_from and y_future >= t + 1:
                            returned_to_origin_within_window = True
                            break
                    if returned_to_origin_within_window:
                        left_temp[(t, c_from, c_to)] += 1
                    else:
                        left_perm[(t, c_from, c_to)] += 1

                    # arrival classification
                    left_destination_within_window = False
                    for j in range(i + 2, n):
                        y_future, c_future = seq[j]
                        if y_future > t + x_perm_years:
                            break
                        if c_future != c_to and y_future >= t + 1:
                            left_destination_within_window = True
                            break
                    if left_destination_within_window:
                        arr_temp[(t, c_from, c_to)] += 1
                    else:
                        arr_perm[(t, c_from, c_to)] += 1

            parsed_rows += 1

        pbar.update(len(chunk))
        del chunk
        gc.collect()

    pbar.close()

    # Build output records for Sankey: keep only Israel as source or target
    records = []
    for key in set(list(left_perm.keys()) + list(left_temp.keys()) +
                   list(arr_perm.keys()) + list(arr_temp.keys())):
        year, source, target = key
        if "Israel" not in (source, target):
            continue
        records.append({
            "year": year,
            "source": source,
            "target": target,
            "left_permanent": left_perm.get(key, 0),
            "left_temporary": left_temp.get(key, 0),
            "arrived_permanent": arr_perm.get(key, 0),
            "arrived_temporary": arr_temp.get(key, 0),
        })

    out_df = pd.DataFrame(records).sort_values(["year", "source", "target"]).reset_index(drop=True)
    out_df.to_csv(output_csv_path, index=False)
    print(f"Done. Parsed rows: {parsed_rows}, skipped rows: {skipped_rows}, output rows: {len(records)}")

    return out_df


# Usage
output_df = compute_sankey_metrics(
    input_csv_path="../data/moving_between_countries/authors_country_changes_in_israel.csv",
    output_csv_path="../data/moving_between_countries/sankey_country_israel.csv",
    x_perm_years=3,
    chunksize=50_000
)


Processing: 109148rows [02:04, 873.52rows/s]


Done. Parsed rows: 109148, skipped rows: 0, output rows: 8325
